In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# AST FINAL

In [2]:
!pip install torchinfo

In [3]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import math
# import os
# import pandas as pd
# from pathlib import Path
# from torchvision import transforms
# from torch.utils.data import Dataset, DataLoader
# from PIL import Image

# # -------------------------
# # Custom Dataset for Spectrograms
# # -------------------------
# class SpectrogramDataset(Dataset):
#     def __init__(self, image_dir, transform=None):
#         self.image_dir = Path(image_dir)
#         self.image_files = [f for f in os.listdir(image_dir) if f.endswith((".png", ".jpg", ".jpeg"))]
#         self.transform = transform

#     def __len__(self):
#         return len(self.image_files)

#     def __getitem__(self, idx):
#         img_file = self.image_files[idx]
#         img_path = self.image_dir / img_file
#         image = Image.open(img_path).convert("L")  # Convert to grayscale
        
#         if self.transform:
#             image = self.transform(image)

#         return image, img_file  # Return image tensor and filename

# # -------------------------
# # Define AST Model (Modified for 3 Classes)
# # -------------------------
# class PatchEmbedding(nn.Module):
#     def __init__(self, img_size=224, patch_size=16, overlap=6, embed_dim=256):
#         super().__init__()
#         self.proj = nn.Conv2d(in_channels=1, out_channels=embed_dim, 
#                               kernel_size=patch_size, stride=patch_size-overlap)

#     def forward(self, x):
#         x = self.proj(x)  # Shape: (batch_size, embed_dim, num_patches_h, num_patches_w)
#         x = x.flatten(2).transpose(1, 2)  # Shape: (batch_size, num_patches, embed_dim)
#         return x

# class MultiHeadSelfAttention(nn.Module):
#     def __init__(self, embed_dim, num_heads):
#         super().__init__()
#         self.num_heads = num_heads
#         self.head_dim = embed_dim // num_heads
#         self.qkv = nn.Linear(embed_dim, embed_dim * 3)
#         self.out = nn.Linear(embed_dim, embed_dim)

#     def forward(self, x):
#         batch_size, num_patches, embed_dim = x.shape
#         qkv = self.qkv(x).reshape(batch_size, num_patches, 3, self.num_heads, self.head_dim)
#         q, k, v = qkv.permute(2, 0, 3, 1, 4)
#         attn_weights = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)
#         attn = F.softmax(attn_weights, dim=-1)
#         out = (attn @ v).transpose(1, 2).reshape(batch_size, num_patches, embed_dim)
#         return self.out(out)

# class TransformerEncoderBlock(nn.Module):
#     def __init__(self, embed_dim, num_heads, mlp_dim, dropout=0.1):
#         super().__init__()
#         self.norm1 = nn.LayerNorm(embed_dim)
#         self.attn = MultiHeadSelfAttention(embed_dim, num_heads)
#         self.norm2 = nn.LayerNorm(embed_dim)
#         self.mlp = nn.Sequential(
#             nn.Linear(embed_dim, mlp_dim),
#             nn.GELU(),
#             nn.Linear(mlp_dim, embed_dim)
#         )
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x):
#         x = x + self.dropout(self.attn(self.norm1(x)))  
#         x = x + self.dropout(self.mlp(self.norm2(x)))   
#         return x

# class AST(nn.Module):
#     def __init__(self, img_size=224, patch_size=16, num_classes=3, embed_dim=256, 
#                  num_heads=8, depth=6, mlp_dim=512, dropout=0.1):
#         super().__init__()
        
#         self.patch_embed = PatchEmbedding(img_size, patch_size, overlap=6, embed_dim=embed_dim)
#         num_patches = self.patch_embed.proj.weight.shape[-1] ** 2
        
#         self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
#         self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, embed_dim))
        
#         self.encoder = nn.Sequential(*[TransformerEncoderBlock(embed_dim, num_heads, mlp_dim, dropout) 
#                                        for _ in range(depth)])
        
#         self.norm = nn.LayerNorm(embed_dim)
#         self.head = nn.Linear(embed_dim, num_classes)

#     def forward(self, x):
#         x = self.patch_embed(x)
#         batch_size = x.shape[0]
#         cls_token = self.cls_token.expand(batch_size, -1, -1)
#         x = torch.cat([cls_token, x], dim=1)
#         x = x + self.pos_embedding
#         x = self.encoder(x)
#         x = self.norm(x[:, 0])  # CLS token output
        
#         soft_labels = torch.sigmoid(self.head(x))  # Multi-label classification
#         return soft_labels


# # -------------------------
# # Load Data and Use DataLoader
# # -------------------------
# # data_dir = Path("your_dataset_directory")  # Change this to your dataset folder
# # train_dir = data_dir / "train"
# # csv_path = data_dir / "train_labels.csv"

# # # Define image transformations
# # transform = transforms.Compose([
# #     transforms.Grayscale(num_output_channels=1),  # Ensure single-channel
# #     transforms.ToTensor(),  # Convert to tensor
# # ])

# # # Create dataset and dataloader
# # batch_size = 32 
# # dataset = SpectrogramDataset(train_dir, transform=transform)
# # dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# # ast_model = AST()
# # ast_model.eval() 

# # # -------------------------
# # # Process Batches Using DataLoader
# # # -------------------------
# # df_records = []

# # with torch.no_grad():
# #     for batch_images, batch_filenames in dataloader:
# #         batch_images = batch_images.unsqueeze(1)  # Add channel dimension (B, 1, H, W)
        
# #         soft_labels = ast_model(batch_images)  # Forward pass
        
# #         for filename, label in zip(batch_filenames, soft_labels):
# #             df_records.append([filename] + label.tolist())

# # # -------------------------
# # # Save Predictions to CSV
# # # -------------------------
# # class_names = ["Class_1", "Class_2", "Class_3"]  # Modify with actual class names
# # df = pd.DataFrame(df_records, columns=["Path"] + class_names)
# # df.to_csv(csv_path, index=False)

# # print(f"Soft labels generated! Labels saved in {csv_path}")


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import os
import pandas as pd
from pathlib import Path
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# -------------------------
# Custom Dataset for Spectrograms
# -------------------------
class SpectrogramDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = Path(image_dir)
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith((".png", ".jpg", ".jpeg"))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_file = self.image_files[idx]
        img_path = self.image_dir / img_file
        image = Image.open(img_path).convert("L")  # Convert to grayscale
        
        if self.transform:
            image = self.transform(image)

        return image, img_file  # Return image tensor and filename

# -------------------------
# Define AST Model 
# -------------------------
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=224, patch_size=16, overlap=6, embed_dim=256):
        super().__init__()
        self.proj = nn.Conv2d(in_channels=1, out_channels=embed_dim, 
                              kernel_size=patch_size, stride=patch_size-overlap)

    def forward(self, x):
        x = self.proj(x)  # Shape: (batch_size, embed_dim, num_patches_h, num_patches_w)
        x = x.flatten(2).transpose(1, 2)  # Shape: (batch_size, num_patches, embed_dim)
        return x

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.qkv = nn.Linear(embed_dim, embed_dim * 3)
        self.out = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        batch_size, num_patches, embed_dim = x.shape
        qkv = self.qkv(x).reshape(batch_size, num_patches, 3, self.num_heads, self.head_dim)
        q, k, v = qkv.permute(2, 0, 3, 1, 4)
        attn_weights = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        attn = F.softmax(attn_weights, dim=-1)
        out = (attn @ v).transpose(1, 2).reshape(batch_size, num_patches, embed_dim)
        return self.out(out)

class TransformerEncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = MultiHeadSelfAttention(embed_dim, num_heads)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_dim),
            nn.GELU(),
            nn.Linear(mlp_dim, embed_dim)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x + self.dropout(self.attn(self.norm1(x)))  
        x = x + self.dropout(self.mlp(self.norm2(x)))   
        return x


class AST(nn.Module):
    def __init__(self, img_size=224, patch_size=16, num_classes=3, embed_dim=256, 
                 num_heads=8, depth=6, mlp_dim=512, dropout=0.1):
        super().__init__()
        
        self.patch_embed = PatchEmbedding(img_size, patch_size, overlap=6, embed_dim=embed_dim)
        
        # Calculate number of patches (H, W)
        num_patches_h = (img_size - patch_size) // (patch_size - 6) + 1
        num_patches_w = (img_size - patch_size) // (patch_size - 6) + 1
        num_patches = num_patches_h * num_patches_w
        
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, embed_dim))  # Fixed here
        
        self.encoder = nn.Sequential(*[TransformerEncoderBlock(embed_dim, num_heads, mlp_dim, dropout) 
                                       for _ in range(depth)])
        
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.patch_embed(x)
        batch_size = x.shape[0]
        cls_token = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat([cls_token, x], dim=1)
        
        # Fix position embedding size
        x = x + self.pos_embedding[:, :x.shape[1], :]
        
        x = self.encoder(x)
        x = self.norm(x[:, 0])  # CLS token output
        
        soft_labels = torch.sigmoid(self.head(x))  # Multi-label classification
        return soft_labels


In [5]:
from torchinfo import summary
ast_model = AST(img_size=224, patch_size=16, num_classes=3, embed_dim=256, 
                num_heads=8, depth=6, mlp_dim=512, dropout=0.1)

device = "cuda" if torch.cuda.is_available() else "cpu"
ast_model.to(device)

summary(ast_model, input_size=(4, 1, 224, 224), col_names=["input_size", "output_size", "num_params"])


Layer (type:depth-idx)                        Input Shape               Output Shape              Param #
AST                                           [4, 1, 224, 224]          [4, 3]                    113,408
├─PatchEmbedding: 1-1                         [4, 1, 224, 224]          [4, 441, 256]             --
│    └─Conv2d: 2-1                            [4, 1, 224, 224]          [4, 256, 21, 21]          65,792
├─Sequential: 1-2                             [4, 442, 256]             [4, 442, 256]             --
│    └─TransformerEncoderBlock: 2-2           [4, 442, 256]             [4, 442, 256]             --
│    │    └─LayerNorm: 3-1                    [4, 442, 256]             [4, 442, 256]             512
│    │    └─MultiHeadSelfAttention: 3-2       [4, 442, 256]             [4, 442, 256]             263,168
│    │    └─Dropout: 3-3                      [4, 442, 256]             [4, 442, 256]             --
│    │    └─LayerNorm: 3-4                    [4, 442, 256]            

In [6]:
#-------------------------
#Load Data and Use DataLoader
#-------------------------
data_dir = Path("your_dataset_directory")  # Change this to your dataset folder
train_dir = data_dir / "train"
csv_path = data_dir / "train_labels.csv"

# Define image transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure single-channel
    transforms.ToTensor(),  # Convert to tensor
])

# Create dataset and dataloader
batch_size = 32 
dataset = SpectrogramDataset(train_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

ast_model = AST()
ast_model.eval() 
# -------------------------
# Process Batches Using DataLoader
# -------------------------
df_records = []

with torch.no_grad():
    for batch_images, batch_filenames in dataloader:
        batch_images = batch_images.unsqueeze(1)  # Add channel dimension (B, 1, H, W)
        
        soft_labels = ast_model(batch_images)  # Forward pass
        
        for filename, label in zip(batch_filenames, soft_labels):
            df_records.append([filename] + label.tolist())

# -------------------------
# Save Predictions to CSV
# -------------------------
class_names = ["Class_1", "Class_2", "Class_3"]  # Modify with actual class names
df = pd.DataFrame(df_records, columns=["Path"] + class_names)
df.to_csv(csv_path, index=False)

print(f"Soft labels generated! Labels saved in {csv_path}")

FileNotFoundError: [Errno 2] No such file or directory: 'your_dataset_directory/train'